# Testing different SA methods 4/5
## Textblob


In [2]:
import csv
import re

from textblob import TextBlob

# Ugly hackery, but necessary: stackoverflow.com/questions/4383571/importing-files-from-different-folder
import sys
sys.path.append('../../../')
    
from src.streaming import spark_functions

In [11]:
preprocess = spark_functions.preprocessor()
tokenize = spark_functions.tokenizer()

with open('./../../../data/interim/sanders_hydrated.csv') as csv_file:
    iterator = csv.reader(csv_file, delimiter=',')
    # Load the parts we need and preprocess as well as tokenize the text
    tweets = [(preprocess(text), sentiment) for (topic, sentiment, id, text) in iterator if sentiment=='positive' or sentiment=='negative']

In [19]:
# TODO Do a "if it gives a polarity at all, it is right in X% of cases"
labeled_correctly = 0
labeled_neutral = 0
for (tweet,sentiment) in tweets:
    analysis = TextBlob(preprocess(tweet)).sentiment
    if (analysis.polarity < 0 and sentiment == 'negative') or (analysis.polarity > 0 and sentiment == 'positive'):
        labeled_correctly += 1
    elif (analysis.polarity == 0):
        labeled_neutral += 1

labeled_not_wrong = labeled_neutral + labeled_correctly
print("Labeled correctly: %d/%d = %.2d percent" % (labeled_correctly, len(tweets), labeled_correctly/len(tweets)*100))
print("Labeled not wrong: %d/%d = %.2d percent" % (labeled_not_wrong, len(tweets), labeled_not_wrong/len(tweets)*100))

Labeled correctly: 456/946 = 48 percent
Labeled not wrong: 738/946 = 78 percent
